In [1]:
# Sprawdzenie dostępności GPU i CUDA
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Włącz GPU dla TensorFlow
        tf.config.experimental.set_memory_growth(gpus[0], True)
        print(f"CUDA jest dostępna! Używam GPU: {gpus[0].name}")
    except RuntimeError as e:
        print(f"Wystąpił problem z konfiguracją GPU: {e}")
else:
    print("CUDA jest niedostępna. Trening będzie przeprowadzony na CPU.")


2024-11-30 00:13:10.302758: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-30 00:13:10.319999: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-30 00:13:10.325183: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


CUDA jest dostępna! Używam GPU: /physical_device:GPU:0


I0000 00:00:1732925592.496141   73405 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1732925592.550125   73405 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1732925592.551683   73405 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355


In [2]:
# Wczytywanie ścieżek z config.json
import json
from pathlib import Path


config_path = Path("../config.json")
with open(config_path, "r") as config_file:
    config = json.load(config_file)

train_file = Path(config["train_file"])
dataset_path = Path(config["dataset_path"])

print(f"Wczytano ścieżki:\nTrain File: {train_file}\nDataset Path: {dataset_path}")


Wczytano ścieżki:
Train File: /home/ubuntu/fashion/fashion/data/styles_train.csv
Dataset Path: /home/ubuntu/.cache/kagglehub/datasets/paramaggarwal/fashion-product-images-small/versions/1


In [3]:
# Wczytywanie zbioru treningowego
import pandas as pd

df_train = pd.read_csv(train_file)

# Tworzenie pełnych ścieżek do obrazów
# Konwersja każdej ścieżki na obiekt Path
df_train["image_path"] = df_train["id"].apply(lambda x: dataset_path / "images" / f"{x}.jpg")
df_train = df_train[df_train["image_path"].apply(lambda path: Path(path).exists())]


# Sprawdzenie danych
print("Pierwsze wiersze zbioru treningowego:")
print(df_train.head())
print(f"Łączna liczba próbek: {len(df_train)}")

Pierwsze wiersze zbioru treningowego:
      id gender masterCategory subCategory   articleType baseColour  season  \
0  17848  women        apparel     topwear       tshirts      white    fall   
1  44045    men  personal care   fragrance         other        red  spring   
2   4006    men       footwear  flip flops    flip flops      black  summer   
3  49978    men        apparel   innerwear         other  navy blue  summer   
4  36220  women       footwear       shoes  sports shoes       blue  summer   

     year   usage                                 productDisplayName  \
0  2011.0  casual        flying machine women mind your white tshirt   
1  2017.0  casual                          hugo men energise perfume   
2  2011.0  casual  spalding men's black yellow with graphic flip ...   
3  2016.0  casual                 hanes men navy blue innerwear vest   
4  2012.0  sports      nike women dual fusion st 2 blue sports shoes   

                                          image_path  

In [4]:
# Ustawienia dla generatora danych
from tensorflow.keras.preprocessing.image import ImageDataGenerator

image_size = (224, 224)  # Rozmiar obrazu zgodny z wejściem ResNet50
batch_size = 32

# Funkcja pomocnicza do ładowania obrazów
def preprocess_image(image_path, label):
    # Wczytanie i przeskalowanie obrazu
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, image_size)
    image = image / 255.0  # Normalizacja do zakresu [0, 1]
    return image, label

# Przygotowanie etykiet
categories = df_train["masterCategory"].unique()
category_to_index = {category: idx for idx, category in enumerate(categories)}
df_train["category_index"] = df_train["masterCategory"].map(category_to_index)

# Konwersja do TensorFlow Dataset
train_ds = tf.data.Dataset.from_tensor_slices(
    (df_train["image_path"].astype(str), df_train["category_index"])
)
train_ds = train_ds.map(preprocess_image).shuffle(1000).batch(batch_size).prefetch(tf.data.AUTOTUNE)


I0000 00:00:1732925593.621979   73405 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1732925593.625197   73405 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1732925593.626707   73405 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1732925593.802935   73405 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

In [5]:
# Wczytanie pretrenowanego modelu ResNet50
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models


base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
base_model.trainable = True  # Odblokuj warstwy
for layer in base_model.layers[:-10]:  # Zamroź większość warstw
    layer.trainable = False

# Dodanie warstw własnych
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(len(categories), activation="softmax")  # Liczba klas zgodna z masterCategory
])

model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 6)              │         1,542 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,113,798 (91.99 MB)

 Trainable params: 4,991,750 (19.04 MB)

 Non-trainable params: 19,122,048 (72.94 MB)

In [6]:
from pathlib import Path

# Sprawdzenie folderu zapisu modelu
models_dir = Path("../models")
models_dir.mkdir(parents=True, exist_ok=True)  # Tworzy katalog, jeśli nie istnieje

# Trening modelu
epochs = 12

history = model.fit(
    train_ds,
    epochs=epochs,
    verbose=1
)


Epoch 1/12


I0000 00:00:1732925601.940812   73508 service.cc:146] XLA service 0x77fef40029c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1732925601.940842   73508 service.cc:154]   StreamExecutor device (0): NVIDIA A100-SXM4-40GB, Compute Capability 8.0
2024-11-30 00:13:22.110935: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-11-30 00:13:23.098898: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 90400
2024-11-30 00:13:23.718843: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_6596', 4 bytes spill stores, 4 bytes spill loads

2024-11-30 00:13:24.114752: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 

   9/1101 ━━━━━━━━━━━━━━━━━━━━ 16s 15ms/step - accuracy: 0.4783 - loss: 1.3954

I0000 00:00:1732925608.186301   73508 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1099/1101 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8495 - loss: 0.4157

2024-11-30 00:13:44.303154: W external/local_tsl/tsl/framework/bfc_allocator.cc:291] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.10GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


1101/1101 ━━━━━━━━━━━━━━━━━━━━ 31s 17ms/step - accuracy: 0.8496 - loss: 0.4154
Epoch 2/12
1101/1101 ━━━━━━━━━━━━━━━━━━━━ 14s 12ms/step - accuracy: 0.9243 - loss: 0.2384
Epoch 3/12
1101/1101 ━━━━━━━━━━━━━━━━━━━━ 14s 12ms/step - accuracy: 0.9350 - loss: 0.2080
Epoch 4/12
1101/1101 ━━━━━━━━━━━━━━━━━━━━ 14s 12ms/step - accuracy: 0.9451 - loss: 0.1806
Epoch 5/12
1101/1101 ━━━━━━━━━━━━━━━━━━━━ 14s 12ms/step - accuracy: 0.9503 - loss: 0.1653
Epoch 6/12
1101/1101 ━━━━━━━━━━━━━━━━━━━━ 14s 12ms/step - accuracy: 0.9514 - loss: 0.1571
Epoch 7/12
1101/1101 ━━━━━━━━━━━━━━━━━━━━ 14s 12ms/step - accuracy: 0.9551 - loss: 0.1519
Epoch 8/12
1101/1101 ━━━━━━━━━━━━━━━━━━━━ 14s 12ms/step - accuracy: 0.9592 - loss: 0.1366
Epoch 9/12
1101/1101 ━━━━━━━━━━━━━━━━━━━━ 14s 12ms/step - accuracy: 0.9600 - loss: 0.1306
Epoch 10/12
1101/1101 ━━━━━━━━━━━━━━━━━━━━ 14s 12ms/step - accuracy: 0.9634 - loss: 0.1267
Epoch 11/12
1101/1101 ━━━━━━━━━━━━━━━━━━━━ 14s 12ms/step - accuracy: 0.9650 - loss: 0.1148
Epoch 12/12
1101/11

In [7]:
# Zapis modelu do folderu ../models/
model_save_path = models_dir / "resnet50_masterCategory_model.h5"
model.save(model_save_path)
print(f"Model został zapisany w: {model_save_path}")

# Aktualizacja pliku config.json
config_path = Path("../config.json")
if config_path.exists():
    with open(config_path, "r") as config_file:
        config = json.load(config_file)
else:
    config = {}

config["resnet50_masterCategory_model"] = str(model_save_path.resolve())

with open(config_path, "w") as config_file:
    json.dump(config, config_file, indent=4)

print(f"Plik konfiguracyjny zaktualizowany: {config_path}")

Model został zapisany w: ../models/resnet50_masterCategory_model.h5
Plik konfiguracyjny zaktualizowany: ../config.json
